# DRL

## 例題
強化学習用環境の1つ「Acrobot-v1」をDQNで学習するプログラムを作成せよ

### STEP1：環境構築とインポート
各種ライブラリをインポートする．

In [ ]:
# OpenAI Gym（強化学習シミュレーション環境）を動かすためのセットアップ
!apt -y install xvfb freeglut3-dev ffmpeg
!pip install gym==0.15.3 pyglet==1.3.2 pyopengl pyvirtualdisplay

# PFRL（Pytorchをラップしている強化学習用ライブラリ）のセットアップ
!pip install pfrl==0.3.0

from IPython.display import clear_output
clear_output()

In [ ]:
# OpenAI Gym
import gym

# 画面表示
import matplotlib.pyplot as plt
import matplotlib.animation
from pyvirtualdisplay import Display
from IPython.display import HTML

# Pytorch関連
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# PFRL関連
from pfrl import action_value
from pfrl import explorers
from pfrl import agents
from pfrl import replay_buffers

# その他
import numpy as np

### STEP2：環境のセットアップ
OpenAI Gymを用いて，強化学習用の環境を準備する．  
今回はAcrobot-v1を用いる．

In [ ]:
# 仮想ディスプレイ
display = Display(visible=0, size=(1024, 768))
display.start()

# OpenAI Gymが提供する環境の準備
# Acrobot-v1: アームの先端が上の線を越えたらクリア
# env.observation_space.shape[0]:観測されるベクトルの大きさ
# env.action_space.n:行動の選択肢の数
env_name = 'Acrobot-v1'
env = gym.make(env_name)
obs_size = env.observation_space.shape[0]
act_size = env.action_space.n

### STEP3：ネットワーク定義
ネットワークを定義するクラスを記述する． 

- クラスはnn.Moduleを継承する
- クラスは```__init__()```と```forward()```というメソッドを定義する
    - ```__init__()```：ネットワークの構造を定義
    - ```forward()```：ネットワークの順伝播を定義
    - 全結合層は```nn.Linear```クラスを用いる
    - 活性化関数ReLUは```F.relu()```を用いる
- `action_value.DiscreteActionValue()`関数を用いて，出力を型変換する

In [ ]:
class Q(nn.Module):
    def __init__(self, obs_size, act_size, hid_size=20):
        super(Q, self).__init__()

        # 全結合層の定義
        # 引数1：入力サイズ
        # 引数2：出力サイズ
        # ---
        # obs_size：観測されるベクトルの大きさ（ネットワークの入力サイズ）
        # hid_size：隠れ層のサイズ
        # act_size：行動の選択肢の数（ネットワークの出力サイズ）
        self.fc1 = nn.Linear(obs_size, hid_size)
        self.fc2 = nn.Linear(hid_size, hid_size)
        self.fc3 = nn.Linear(hid_size, act_size)

    def forward(self, x):

        # 全結合層の順伝播＋活性化関数ReLU
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)

        # 型変換する
        x = action_value.DiscreteActionValue(x)

        return x

In [ ]:
# 上記Qクラスのインスタンスを生成
q = Q(obs_size, act_size)

# 最適化器（確率的勾配降下法）の定義
# 引数1：最適化するネットワークのパラメータ parameters()で取得可能
# 引数2：学習率 = 0.01
optimizer = optim.SGD(q.parameters(), lr=0.01)

### STEP4：Agentの用意
Agentを用意する．

In [ ]:
# 割引率
gamma = 0.99

# εグリーディー法：確率εと探索行動（ランダムに行動）を指定
exp = explorers.ConstantEpsilonGreedy(
   epsilon=0.25,
   random_action_func=env.action_space.sample
)

# Experience Replay：蓄積する量を指定
rep = replay_buffers.ReplayBuffer(capacity=10**6)

# 観測データをnp.float32に変換
phi = lambda x: x.astype(np.float32, copy=False)

# DQNのAgentを準備
age = agents.DQN(
    q, optimizer, rep, gamma, exp,
    replay_start_size=500,       # Experienceがこれだけ蓄積してから学習
    update_interval=1,           # 更新頻度
    target_update_interval=500,  # targetに含まれるQ値の更新頻度
    phi=phi
)

### STEP5：ネットワーク学習
下記のステップで行う．
- `age.act()`: 行動を選択する
- `env.step()`: 選択した行動を環境へ入力し，結果を得る
- `age.observe()`: 行動した結果をAgentへ与える

In [ ]:
# エピソードを何回繰り返すか
# エピソード = 環境を初期化してからゲームオーバーするまで
EPISODES = 20

# 1エピソードの長さをどうするか
EPISODE_TIME = 500

# 学習
episodes = []
for episode in range(EPISODES):
    obs = env.reset()

    done = False
    r = 0
    R = 0
    t = 0

    # ゲームオーバーかタイムアップまで繰り返す
    frames = []
    while (not done) and (t < EPISODE_TIME):
        # 画面出力
        frames.append(env.render(mode='rgb_array'))

        # 学習，行動と観測        
        action = age.act(obs) # 学習と行動の選択
        obs, r, done, _ = env.step(action) # 実際に行動を行う
        R += r # 得られた報酬の和を計算
        t += 1
        reset = t == EPISODE_TIME
        age.observe(obs, r, done, reset) # 行動結果の観測

    # エピソード終了
    print('episode', episode, 'sum of reward', R)
    episodes.append(frames)

In [ ]:
# 保存したアニメーションを表示
episode = 19
fig = plt.figure()
plt.axis('off')
ims = []
for frame in episodes[episode]:
    im = plt.imshow(frame)
    ims.append([im])
ani = matplotlib.animation.ArtistAnimation(fig, ims, interval=30)
HTML(ani.to_jshtml())

## 演習
強化学習用環境の1つ「CartPole-v1」をDQNで学習するプログラムを作成せよ

### STEP1：環境構築とインポート
各種ライブラリをインポートする．

In [ ]:
# OpenAI Gym（強化学習シミュレーション環境）を動かすためのセットアップ
!apt -y install xvfb freeglut3-dev ffmpeg
!pip install gym==0.15.3 pyglet==1.3.2 pyopengl pyvirtualdisplay

# PFRL（Pytorchをラップしている強化学習用ライブラリ）のセットアップ
!pip install pfrl==0.3.0

from IPython.display import clear_output
clear_output()

### STEP2：環境のセットアップ
OpenAI Gymを用いて，強化学習用の環境を準備する．  
今回はCartPole-v1を用いる．

### STEP3：ネットワーク定義

### STEP4：Agentの用意
Agentを用意する．

### STEP5：ネットワーク学習